## DuckDB Notebook

This notebook generates a bunch of raw outputs, without applying PAC, to be consumed by a second stage.

```
 group by              output cols                        
 key cols ┌────────┬────────┬────────┬────────┐           
        │ │   A    │   B    │   C    │   D    │           
      ┌─▼─┼────────┼────────┼────────┼────────┤           
      │ 1 │   2    │        │        │        │           
      ├───┼───|────┼────────┼────────┼────────┤           
      │ 2 │   │    │        │        │        │           
      ├───┼───┼────┼────────┼────────┼────────┤           
      │ 3 │   │    │        │        │        │           
      └───┴───┼────┴────────┴────────┴────────┘           
              ▼                 A_1.json                  
       Sample 0:   A1=2        ┌─────────────────────────┐
       Sample 1:   A1=4  ───▶  │{                        │
             ...               │    col: A               │
       Sample 999: A1=3        │    row: 1               │
                               │    value: [2, 4, ... 3] │
                               │}                        │
                               └─────────────────────────┘
```

In [ ]:
#!/usr/bin/env python
# coding: utf-8

EXPERIMENT = 'pac-duckdb-q1'
OUTPUT_DIR = f'./outputs/{EXPERIMENT}-step1'

import os
from typing import List
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

import numpy as np
import pickle
import json
import duckdb
import polars as pl
import pyarrow as pa
import shutil

In [2]:
# duckdb load data/tpch/tpch.duckdb into the temporary in-memory database
con = duckdb.connect(database=':memory:')
tables = ["customer", "lineitem", "nation", "orders", "part", "partsupp", "region", "supplier"]
for t in tables:
    con.execute(f"CREATE TABLE {t} AS SELECT * FROM 'data/tpch/{t}.parquet'")

In [3]:
# Construct the table of random samples
# to use, join it with the lineitem table (for specific sample # s) and filter to just the
# rows where random_binary = 1.0
# This will give us a 50% sample of the lineitem table for each sample # s

SAMPLES = 1024
assert SAMPLES % 2 == 0, "SAMPLES must be even to create complementary samples."

random_samples = con.execute(f"""
DROP TABLE IF EXISTS random_samples;

CREATE TABLE random_samples AS
WITH sample_numbers AS MATERIALIZED (
    SELECT range AS sample_id FROM range({SAMPLES//2})
), random_values AS MATERIALIZED (
    SELECT 
        sample_numbers.sample_id,
        customer.rowid AS row_id,
        (RANDOM() > 0.5)::BOOLEAN AS random_binary
    FROM sample_numbers
    JOIN customer ON TRUE  -- Cross join to duplicate rows for each sample
)
SELECT
    sample_id,
    row_id,
    random_binary
FROM random_values
UNION ALL
SELECT -- select the complementary samples too
    ({SAMPLES//2}) + sample_id,
    row_id,
    NOT random_binary  -- Inverse the random_binary to get the complementary sample
FROM random_values
ORDER BY sample_id, row_id;
""")

The randomness of what rows are chosen is saved to disk in `random_binary.json`. For each sample #, there is an array with one entry per row, where 1 means the row was chosen and 0 means it was not.

In [4]:
con.execute(f"""
SELECT sample_id, array_agg(random_binary::TINYINT) as random_binary
FROM random_samples
GROUP BY sample_id;
""").pl().write_json(f"{OUTPUT_DIR}/random_binary.json")

Query is specified as a prepared statement. We will then execute it once per sample.

In [5]:
# Query
con.execute("""
DEALLOCATE PREPARE run_query;

PREPARE run_query AS 
SELECT
    l_returnflag,
    l_linestatus,
    2*sum(l_quantity) AS sum_qty,
    2*sum(l_extendedprice) AS sum_base_price,
    2*sum(l_extendedprice * (1 - l_discount)) AS sum_disc_price,
    2*sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge,
    avg(l_quantity) AS avg_qty,
    avg(l_extendedprice) AS avg_price,
    avg(l_discount) AS avg_disc,
    2*count(*) AS count_order
FROM
    lineitem
JOIN orders ON lineitem.l_orderkey = orders.o_orderkey
JOIN customer ON orders.o_custkey = customer.c_custkey
JOIN random_samples AS rs
    ON rs.row_id = customer.rowid
WHERE
    l_shipdate <= CAST('1998-09-02' AS date)
    AND rs.random_binary = TRUE
    AND rs.sample_id = $sample
GROUP BY
    l_returnflag,
    l_linestatus
ORDER BY
    l_returnflag,
    l_linestatus;
""")

# Run query to see output
dfs0 = con.execute(f"EXECUTE run_query(sample := {0});").pl()

# Save csv copies of the first 5 samples
os.makedirs(f"{OUTPUT_DIR}/csv", exist_ok=True)
for s in range(5):
    con.execute(f"EXECUTE run_query(sample := {s});").pl().write_csv(f"{OUTPUT_DIR}/csv/sample_{s}.csv")

dfs0

l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
str,str,"decimal[38,2]","decimal[38,2]","decimal[38,4]","decimal[38,6]",f64,f64,f64,i64
"""A""","""F""",3790112.00,5343302545.20,5075830326.8376,5278929038.269122,25.558094,36031.818854,0.050038,148294
"""N""","""F""",98118.00,137425236.14,130639084.6582,135777265.079778,25.445539,35639.324725,0.049362,3856
"""N""","""O""",7530256.00,10610500466.18,10078939700.8858,10481994797.255912,25.571019,36030.821594,0.050133,294484
"""R""","""F""",3786362.00,5337117579.72,5071651650.7752,5274130172.139980,25.486403,35924.703022,0.049908,148564


In [6]:
# Run the query for each sample, but accumulate in a pl.DataFrame instead of a list
dfsdf: pl.DataFrame = pl.concat(
    con.execute(f"EXECUTE run_query(sample := {s});").pl().insert_column(0, pl.lit(s).alias("sample"))
    for s in range(SAMPLES)
)
dfsdf

sample,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
i32,str,str,"decimal[38,2]","decimal[38,2]","decimal[38,4]","decimal[38,6]",f64,f64,f64,i64
0,"""A""","""F""",3790112.00,5343302545.20,5075830326.8376,5278929038.269122,25.558094,36031.818854,0.050038,148294
0,"""N""","""F""",98118.00,137425236.14,130639084.6582,135777265.079778,25.445539,35639.324725,0.049362,3856
0,"""N""","""O""",7530256.00,10610500466.18,10078939700.8858,10481994797.255912,25.571019,36030.821594,0.050133,294484
0,"""R""","""F""",3786362.00,5337117579.72,5071651650.7752,5274130172.139980,25.486403,35924.703022,0.049908,148564
1,"""A""","""F""",3771282.00,5320800911.22,5054843348.2914,5257691362.812702,25.500588,35978.097986,0.049998,147890
…,…,…,…,…,…,…,…,…,…,…
1022,"""R""","""F""",3730456.00,5258082758.70,4996022601.2144,5195506143.018882,25.481953,35916.847173,0.049908,146396
1023,"""A""","""F""",3790652.00,5344112732.70,5076373950.2794,5279719658.274210,25.552431,36024.164348,0.050112,148348
1023,"""N""","""F""",98464.00,137914255.52,130957761.8654,136283765.522668,25.325103,35471.773539,0.050273,3888


In [7]:
# Define which columns are the group-by keys (INDEX_COLS) and which are the output columns (OUTPUT_COLS)
INDEX_COLS = ['l_returnflag', 'l_linestatus']
OUTPUT_COLS = ['sum_qty', 'sum_base_price', 'sum_disc_price', 'sum_charge', 'avg_qty', 'avg_price', 'avg_disc', 'count_order']

# Save these to disk for later use
with open(f"{OUTPUT_DIR}/INDEX_COLS.pkl", 'wb') as f:
    pickle.dump(INDEX_COLS, f)
with open(f"{OUTPUT_DIR}/OUTPUT_COLS.pkl", 'wb') as f:
    pickle.dump(OUTPUT_COLS, f)

In [8]:
# Combine all the samples into one table, grouped-by the group-by keys. Each cell contains an n <= # of samples length array of values.
listdf = (dfsdf.group_by(INDEX_COLS, maintain_order=True)
      .all()
      .drop(pl.col("sample")))
listdf

l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
str,str,"list[decimal[38,2]]","list[decimal[38,2]]","list[decimal[38,4]]","list[decimal[38,6]]",list[f64],list[f64],list[f64],list[i64]
"""A""","""F""","[3790112.00, 3771282.00, … 3790652.00]","[5343302545.20, 5320800911.22, … 5344112732.70]","[5075830326.8376, 5054843348.2914, … 5076373950.2794]","[5278929038.269122, 5257691362.812702, … 5279719658.274210]","[25.558094, 25.500588, … 25.552431]","[36031.818854, 35978.097986, … 36024.164348]","[0.050038, 0.049998, … 0.050112]","[148294, 147890, … 148348]"
"""N""","""F""","[98118.00, 98404.00, … 98464.00]","[137425236.14, 137461788.98, … 137914255.52]","[130639084.6582, 130792383.2078, … 130957761.8654]","[135777265.079778, 136034319.670100, … 136283765.522668]","[25.445539, 25.493264, … 25.325103]","[35639.324725, 35611.86243, … 35471.773539]","[0.049362, 0.048746, … 0.050273]","[3856, 3860, … 3888]"
"""N""","""O""","[7530256.00, 7585948.00, … 7483722.00]","[10610500466.18, 10693069371.90, … 10546463371.98]","[10078939700.8858, 10157246685.1930, … 10018252510.6888]","[10481994797.255912, 10563374792.711254, … 10417896381.538192]","[25.571019, 25.577394, … 25.607611]","[36030.821594, 36053.614347, … 36087.62266]","[0.050133, 0.050195, … 0.050065]","[294484, 296588, … 292246]"
"""R""","""F""","[3786362.00, 3811034.00, … 3837246.00]","[5337117579.72, 5373813436.18, … 5408584215.06]","[5071651650.7752, 5106017544.8474, … 5138836850.8116]","[5274130172.139980, 5310132175.427302, … 5343600706.665366]","[25.486403, 25.528067, … 25.544515]","[35924.703022, 35996.285275, … 36004.900978]","[0.049908, 0.049992, … 0.049952]","[148564, 149288, … 150218]"


In [9]:
# What are all the possible group-by key combinations?
allgroups: pl.DataFrame = listdf.select(INDEX_COLS)
allgroups.to_dicts()

[{'l_returnflag': 'A', 'l_linestatus': 'F'},
 {'l_returnflag': 'N', 'l_linestatus': 'F'},
 {'l_returnflag': 'N', 'l_linestatus': 'O'},
 {'l_returnflag': 'R', 'l_linestatus': 'F'}]

In [10]:
# Template for the final output, including all possible group-by groups
# Obtained by collecting all the samples in a big table and then keeping only the first occurrence of each groupby key.
# Then, fill all OUTPUT_COLS with nulls
templatedf = dfsdf.drop(pl.col("sample")).group_by(INDEX_COLS, maintain_order=True).first()
templatedf = templatedf.clear(n=len(allgroups)).with_columns(allgroups)
templatedf

with open(f"{OUTPUT_DIR}/template.pkl", "wb") as f:
    pickle.dump(templatedf, f)

In [11]:
# Write all table entries in the output table to their own JSON files. Each file has a number, the information of which file corresponds to which table entry
# is stored in reverse_map.json (as well as in the files themselves)
os.makedirs(f"{OUTPUT_DIR}/json", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/pkl", exist_ok=True)
i: int = 0
reverse_map = {}
for col in OUTPUT_COLS:
    for group in allgroups.iter_rows(named=True):
        values = listdf.filter(pl.col(k).eq(v) for k, v in group.items()).select(col).to_series()
        j = pl.DataFrame().with_columns([
            pl.lit(col).alias("col"),
            pl.lit(group).alias("row"),
            pl.lit(values.explode().dtype.__repr__()).alias("dtype"),
            values.alias("values"),
        ])
        reverse_map[i] = (col, group)
        j.write_json(f"{OUTPUT_DIR}/json/{i}.json")
        # save pickle of numpy conversion of values
        pickle.dump(values.explode().to_numpy(), open(f"{OUTPUT_DIR}/pkl/{i}.pkl", "wb"))
        i+=1

with open(f"{OUTPUT_DIR}/reverse_map.json", "w") as f:
    json.dump(reverse_map, f)
pickle.dump(reverse_map, open(f"{OUTPUT_DIR}/reverse_map.pkl", "wb"))

In [ ]:
# zip the OUTPUT_DIR
shutil.make_archive(OUTPUT_DIR, 'zip', OUTPUT_DIR)

'/Users/michael/projects/dpdb/pacdb/outputs/pac-duckdb-q1-step1.zip'